# =========================
# 04_yolov8_detection.ipynb
# YOLOv8 object detection for Bird vs Drone (Aerial)
# =========================

# Cell 1: Imports & environment check
import os
import yaml
from pathlib import Path
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

# Check ultralytics
try:
    from ultralytics import YOLO
    print("Ultralytics YOLO imported. Version:", YOLO.__version__ if hasattr(YOLO, '__version__') else 'unknown')
except Exception as e:
    print("Ultralytics not installed or import failed:", e)
    print("Install with: pip install ultralytics")

# Cell 2: Inspect uploaded project spec (your uploaded file)
spec_path = "/mnt/data/detection.txt"   # path to the uploaded file (project spec)

print("Spec file path:", spec_path)
if os.path.exists(spec_path):
    print("\n--- first 2000 characters of the spec file ---\n")
    with open(spec_path, "r", encoding="utf-8", errors="ignore") as f:
        print(f.read()[:2000])
else:
    print("Spec file not found at the path above. Make sure the file exists.")

# Cell 3: Prepare dataset folders (if not already present)
# Expected structure:
# data/object_detection_dataset/
#   images/train, images/val, images/test
#   labels/train, labels/val, labels/test

base = Path("data/object_detection_dataset")
images = base / "images"
labels = base / "labels"

for sub in ["train", "val", "test"]:
    (images / sub).mkdir(parents=True, exist_ok=True)
    (labels / sub).mkdir(parents=True, exist_ok=True)

print("Object detection dataset directories ensured under:", base.resolve())
print("Images folders:", list((images).glob("*")))
print("Labels folders:", list((labels).glob("*")))

# Cell 4: Create data.yaml for YOLOv8
# This file tells Ultralytics where the images/labels are and what the classes are.

data_yaml = {
    "path": str(base.resolve()),          # base path for images/labels
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "nc": 2,
    "names": ["bird", "drone"]
}

yaml_path = base / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

print("Created data.yaml at:", yaml_path)
print("--- data.yaml content ---")
print(yaml.dump(data_yaml, default_flow_style=False))

# Cell 5: Notes on preparing labels
print("""
Label format required by YOLOv8 (one .txt per image, same basename as image).
Each line in label .txt:
<class_id> <x_center> <y_center> <width> <height>

All coordinates must be normalized (0..1 relative to image width/height).

Examples:
- If you have Pascal VOC or COCO, convert annotations to YOLO text format.
- If you only have bounding boxes stored elsewhere, write a small converter to YOLO format.

Place .jpg/.png images in:
  data/object_detection_dataset/images/train  (and val, test)

Place .txt label files in corresponding:
  data/object_detection_dataset/labels/train  (and val, test)
""")

# Cell 6: Quick helper - list a few sample images and labels if present
def list_samples(n=5):
    train_imgs = list((images / "train").glob("*"))
    print("Train images found:", len(train_imgs))
    for p in train_imgs[:n]:
        print("-", p.name)
    train_lbls = list((labels / "train").glob("*"))
    print("Train labels found:", len(train_lbls))
    for p in train_lbls[:n]:
        print("-", p.name)

list_samples()
